In [20]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [21]:
from langchain_teddynote import logging

logging.langsmith('langchain_study_2024')

LangSmith 추적을 시작합니다.
[프로젝트명]
langchain_study_2024


In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores.faiss import FAISS

In [23]:
# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [24]:
# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

In [25]:
# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings = embeddings,
    document_embedding_cache = store,
    namespace=embeddings.model,         # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [26]:
# store에서 키들을 순차적으로 가져옵니다.
list(store.yield_keys())

[]

In [27]:
# 문서를 로드하고, 청크로 분할한 다음, 각 청크를 임베딩하고 벡터 저장소에 로드합니다.

In [28]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

In [29]:
# 문서 로드
raw_documents = TextLoader("./data/book_document.txt", encoding='utf-8').load()

In [30]:
# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [31]:
# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [32]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: total: 328 ms
Wall time: 1.14 s


In [33]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 0 ns
Wall time: 34.4 ms
